# Oanda Error Debuging

In [2]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timedelta, timezone # Need to import timezone seperatly
import time

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [14]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = [] 
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        print("\n--- Entering get_most_recent ---") # DEBUG
        loop_counter = 0 # DEBUG counter
        max_loops = 10 # DEBUG limit

        while True:
            loop_counter += 1 # DEBUG
            print(f"\nLoop Iteration: {loop_counter}") # DEBUG

            if loop_counter > max_loops: # DEBUG safety break
                print(f"DEBUG: Exiting loop after {max_loops} iterations to prevent infinite run.")
                break

            time.sleep(2)
            # Keep using naive UTC for loop checks
            now_naive = datetime.now(timezone.utc).replace(tzinfo=None)
            now_naive = now_naive - timedelta(microseconds = now_naive.microsecond)
            past_naive = now_naive - timedelta(days = days) # Use naive past for query if API accepts it

            # It's often safer to query API with aware UTC if it expects it
            now_aware = datetime.now(timezone.utc)
            past_aware = now_aware - timedelta(days = days)

            print(f"Fetching history from {past_aware} to {now_aware}") # DEBUG API call times

            try:
                # get_history likely returns TZ-AWARE index (UTC)
                df = self.get_history(instrument = self.instrument, start = past_aware, end = now_aware,
                                      granularity = "S5", price = "M", localize = False).c.dropna().to_frame()


                if df.empty:
                    print("Warning: get_history returned empty DataFrame. Waiting and retrying.") # DEBUG
                    continue # Skip rest of loop and try again

                df.rename(columns = {"c":self.instrument}, inplace = True)
                print(f"  History data shape BEFORE resample: {df.shape}")
                if not df.empty:
                    print(f"  History data index TZ before resample: {df.index.tz}") # DEBUG Check TZ
                    print(f"  History data tail BEFORE resample:\n{df.tail(3)}")

                # Resampling likely keeps index TZ-AWARE
                df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1] # Keep iloc[-1] for now as you intended

                # Debug print corrections from before:
                print(f"  Resampled data shape (after dropna, iloc): {df.shape}")
                if df.empty:
                    print("Warning: Resampled DataFrame is empty after dropna/iloc. Waiting and retrying.")
                    continue
                print(f"  Resampled data index TZ after resample: {df.index.tz}") # DEBUG Check TZ
                print(f"  Resampled data tail:\n{df.tail(3)}")


                self.raw_data = df.copy() # raw_data index is TZ-AWARE

                # --- FIX: Convert last_bar to NAIVE UTC ---
                if not self.raw_data.empty:
                    aware_last_bar = self.raw_data.index[-1] # This is TZ-AWARE
                    self.last_bar = aware_last_bar.tz_localize(None) # Convert to NAIVE UTC
                    print(f"DEBUG: Initial self.last_bar set to NAIVE UTC: {self.last_bar}")
                else:
                    # Handle case where df might become empty after resampling/iloc
                    print("WARNING: raw_data is empty after resample/iloc, cannot set last_bar. Retrying.")
                    continue


                # --- FIX: Use consistent NAIVE times for loop check ---
                current_check_time_naive = datetime.now(timezone.utc).replace(tzinfo=None)
                if self.last_bar is None: # Should not happen if code above worked, but safety check
                    print("WARNING: self.last_bar is None before time diff check. Retrying.")
                    continue

                time_diff = current_check_time_naive - self.last_bar # NAIVE - NAIVE = OK

                print(f"  Current Naive UTC Time: {current_check_time_naive}")
                print(f"  Last Bar Time (Naive UTC): {self.last_bar} (Type: {type(self.last_bar)})")
                print(f"  Bar Length:         {self.bar_length} (Type: {type(self.bar_length)})")
                print(f"  Time Difference:    {time_diff}")
                print(f"  Condition Check: (Time Diff < Bar Length?) : {time_diff < self.bar_length}")

                if time_diff < self.bar_length:
                    print("--- Exit condition met. Breaking loop. ---") # DEBUG
                    break # Exit the loop

                # Remove the potentially problematic redundant check:
                # if pd.to_datetime(datetime.now(timezone.utc)) - self.last_bar < self.bar_length:
                #    break

            except Exception as e:
                print(f"ERROR during get_history or processing: {e}") # DEBUG
                import traceback # Import traceback for detailed error
                traceback.print_exc() # Print full traceback for debugging
                print("Waiting before retry...")
                time.sleep(5)

        print("--- Exiting get_most_recent ---") # DEBUG        
            
                
                
    def start_trading(self, days): # NEW
        
        self.get_most_recent(days)
        self.stream_data(self.instrument)
            
    def on_success(self, time, bid, ask):
        print(self.ticks, end = "\r", flush = True)
        
        # recent_tick = pd.to_datetime(time)
        try:
            # Attempt conversion assuming 'time' might be tz-aware
            recent_tick = pd.to_datetime(time).tz_convert('UTC').tz_localize(None)
        except TypeError:
            # If 'time' is already naive (or conversion fails), assume it's UTC
            recent_tick = pd.to_datetime(time) # Already naive
        
        # define stop
        if self.ticks >= 100:
            self.terminate_session(cause = "Scheduled Session End.")
            return
        
        # collect and store tick data
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = pd.concat([self.tick_data, df]) 
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar >= self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()

        if recent_tick - self.last_bar >= self.bar_length:
            # DEBUG: Log when resampling is triggered
            print(f"\nDEBUG: Triggering resample. Recent Tick: {recent_tick}, Last Bar: {self.last_bar}, Diff: {recent_tick - self.last_bar}")
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
            
    def resample_and_join(self):
        # Resampling tick_data (which has naive index from on_success)
        # label='right' should make the new index naive as well if the source is naive.
        # Let's verify.
        resampled_ticks = self.tick_data.resample(self.bar_length,
                                                  label="right").last().ffill().iloc[:-1] # Keeping iloc[-1]
        print(f"DEBUG: Resampled ticks index TZ: {resampled_ticks.index.tz}") # Check TZ

        # Concatenating aware raw_data with naive resampled_ticks might cause issues or auto-conversion.
        # It's safer to make raw_data naive *before* concat or make resampled_ticks aware.
        # Let's try making raw_data naive first.

        if self.raw_data.index.tz is not None:
            print("DEBUG: Converting raw_data index to naive before concat.")
            self.raw_data.index = self.raw_data.index.tz_localize(None)

        # Now concat naive with naive
        self.raw_data = pd.concat([self.raw_data, resampled_ticks])

        # Ensure no duplicate indices if resampling overlaps - keep last entry
        self.raw_data = self.raw_data[~self.raw_data.index.duplicated(keep='last')]


        # --- FIX: Ensure last_bar is NAIVE UTC ---
        if not self.raw_data.empty:
            new_last_bar = self.raw_data.index[-1]
            # It *should* be naive now after the concat logic above, but double-check/enforce
            if hasattr(new_last_bar, 'tzinfo') and new_last_bar.tzinfo is not None:
                 print("WARNING: last_bar was still aware after concat, converting.")
                 self.last_bar = new_last_bar.tz_localize(None) # Convert to NAIVE UTC
            else:
                 self.last_bar = new_last_bar # Already naive

            print(f"DEBUG: Updated self.last_bar (Naive UTC): {self.last_bar}")

            # Reset tick_data: Keep only ticks AFTER the new naive last bar time
            self.tick_data = self.tick_data[self.tick_data.index > self.last_bar]
        else:
             # Handle case where raw_data might become empty
             print("WARNING: raw_data became empty during resample_and_join.")
             self.last_bar = None # Or handle differently
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
        
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")  
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")  
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")  
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")  
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")  
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL")  
            self.position = 0
    
    def report_trade(self, order, going):  
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  
        
    def terminate_session(self, cause): # NEW
        self.stop_stream = True
        if self.position != 0:
            close_order = self.create_order(self.instrument, units = -self.position * self.units,
                                            suppress = True, ret = True) 
            self.report_trade(close_order, "GOING NEUTRAL")
            self.position = 0
        print(cause)

In [16]:
trader = ConTrader("oanda.cfg", "EUR_USD", "1min", window = 1, units = 100)

In [22]:
# This execution code needs to replaced.

# trader.get_most_recent()
# trader.stream_data(trader.instrument, stop = 100)
# if trader.position != 0:
#     close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units,
#                                       suppress = True, ret = True) 
#     trader.report_trade(close_order, "GOING NEUTRAL")
#     trader.position = 0

In [24]:
trader.start_trading(days = 5)


--- Entering get_most_recent ---

Loop Iteration: 1
Fetching history from 2025-03-26 06:33:18.016485+00:00 to 2025-03-31 06:33:18.016485+00:00
ERROR during get_history or processing: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-03-26T06%3A33%3A18.016485%2B00%3A00.000000000Z&to=2025-03-26T12%3A33%3A18.016485%2B00%3A00.000000000Z does not have field 'candles' (contains 'errorMessage')
Waiting before retry...


Traceback (most recent call last):
  File "C:\Users\audri\AppData\Local\Temp\ipykernel_24564\4221999248.py", line 45, in get_most_recent
    df = self.get_history(instrument = self.instrument, start = past_aware, end = now_aware,
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 232, in get_history
    batch = self.retrieve_data(instrument, batch_start, batch_end,
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 167, in retrieve_data
    raw = raw.get('candles')
          ^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\v20\response.py", line 35, in get
    raise ResponseNoField(self, field)
v20.errors.ResponseNoField: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-03-26T06%3A33%3A18.0164


Loop Iteration: 2
Fetching history from 2025-03-26 06:33:25.155227+00:00 to 2025-03-31 06:33:25.155227+00:00
ERROR during get_history or processing: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-03-26T06%3A33%3A25.155227%2B00%3A00.000000000Z&to=2025-03-26T12%3A33%3A25.155227%2B00%3A00.000000000Z does not have field 'candles' (contains 'errorMessage')
Waiting before retry...


Traceback (most recent call last):
  File "C:\Users\audri\AppData\Local\Temp\ipykernel_24564\4221999248.py", line 45, in get_most_recent
    df = self.get_history(instrument = self.instrument, start = past_aware, end = now_aware,
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 232, in get_history
    batch = self.retrieve_data(instrument, batch_start, batch_end,
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 167, in retrieve_data
    raw = raw.get('candles')
          ^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\v20\response.py", line 35, in get
    raise ResponseNoField(self, field)
v20.errors.ResponseNoField: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-03-26T06%3A33%3A25.1552


Loop Iteration: 3
Fetching history from 2025-03-26 06:33:32.269827+00:00 to 2025-03-31 06:33:32.269827+00:00
ERROR during get_history or processing: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-03-26T06%3A33%3A32.269827%2B00%3A00.000000000Z&to=2025-03-26T12%3A33%3A32.269827%2B00%3A00.000000000Z does not have field 'candles' (contains 'errorMessage')
Waiting before retry...


...

Traceback (most recent call last):
  File "C:\Users\audri\AppData\Local\Temp\ipykernel_24564\4221999248.py", line 45, in get_most_recent
    df = self.get_history(instrument = self.instrument, start = past_aware, end = now_aware,
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 232, in get_history
    batch = self.retrieve_data(instrument, batch_start, batch_end,
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 167, in retrieve_data
    raw = raw.get('candles')
          ^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\v20\response.py", line 35, in get
    raise ResponseNoField(self, field)
v20.errors.ResponseNoField: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-03-26T06%3A34%3A15.0073


Loop Iteration: 10
Fetching history from 2025-03-26 06:34:22.141921+00:00 to 2025-03-31 06:34:22.141921+00:00
ERROR during get_history or processing: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-03-26T06%3A34%3A22.141921%2B00%3A00.000000000Z&to=2025-03-26T12%3A34%3A22.141921%2B00%3A00.000000000Z does not have field 'candles' (contains 'errorMessage')
Waiting before retry...


Traceback (most recent call last):
  File "C:\Users\audri\AppData\Local\Temp\ipykernel_24564\4221999248.py", line 45, in get_most_recent
    df = self.get_history(instrument = self.instrument, start = past_aware, end = now_aware,
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 232, in get_history
    batch = self.retrieve_data(instrument, batch_start, batch_end,
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 167, in retrieve_data
    raw = raw.get('candles')
          ^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\v20\response.py", line 35, in get
    raise ResponseNoField(self, field)
v20.errors.ResponseNoField: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-03-26T06%3A34%3A22.1419


Loop Iteration: 11
DEBUG: Exiting loop after 10 iterations to prevent infinite run.
--- Exiting get_most_recent ---


TypeError: unsupported operand type(s) for -: 'Timestamp' and 'NoneType'

### Added Wrapper methodes for start and terminate Trading Sessions